In [16]:
import time
from __future__ import print_function


for x in range(20):
    print('Epoch {}, Loss: {:.2f}\r'.format(x, x*2), end='')
    time.sleep(0.1)
print()
print('a')

Epoch 19, Loss: 38.00
a
